# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,anadyr,64.7500,177.4833,10.76,85,98,2.24,RU,1713816165
1,1,port-aux-francais,-49.3500,70.2167,42.96,78,98,24.79,TF,1713816165
2,2,kondoa,-4.9000,35.7833,63.82,96,100,3.38,TZ,1713816165
3,3,adamstown,-25.0660,-130.1015,72.57,76,74,25.75,PN,1713816165
4,4,albany,42.6001,-73.9662,50.14,32,0,8.84,US,1713816165


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', size='Humidity', hover_cols=['City', 'Humidity'],
                                       title='City Humidity', frame_height=400, frame_width=700)

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] < 80) & 
                                (city_data_df['Max Temp'] > 70) & 
                                (city_data_df['Wind Speed'] < 10) & 
                                (city_data_df['Cloudiness'] == 0)]


# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()


# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,sao fidelis,-21.6461,-41.7469,76.51,82,0,1.83,BR,1713816167
165,165,toliara,-23.3500,43.6667,71.40,71,0,9.22,MG,1713816195
185,185,turpan,42.9333,89.1667,71.87,15,0,2.73,CN,1713816199
218,218,gebeit,21.0667,36.3167,73.54,25,0,6.38,SD,1713816206
330,330,prado,-17.3411,-39.2208,79.65,79,0,5.84,BR,1713816227


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,sao fidelis,BR,-21.6461,-41.7469,82,
165,toliara,MG,-23.3500,43.6667,71,
185,turpan,CN,42.9333,89.1667,15,
218,gebeit,SD,21.0667,36.3167,25,
330,prado,BR,-17.3411,-39.2208,79,
372,parral,MX,26.9333,-105.6667,29,
413,rakops,BW,-21.0167,24.3333,25,
475,new mirpur,PK,33.1418,73.7674,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000  # Radius in meters
params = {
    "categories": "accommodation",  # Category for accommodation
    "limit": 1,  # Limit the number of results to 1
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lat},{lng}"  # Filter within 10,000 meters of the city's coordinates
    params["bias"] = f"point:{lat},{lng}"  # Bias towards the city's coordinates
    
    # Construct the API request URL
    api_url = f"{base_url}?lat={lat}&lon={lng}&categories={params['categories']}&apiKey={params['apiKey']}"
    
    # Make an API request
    response = requests.get(api_url)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sao fidelis - nearest hotel: Hotel São José
toliara - nearest hotel: Ambary
turpan - nearest hotel: Oriental
gebeit - nearest hotel: Port Sudan
prado - nearest hotel: Pousada Recanto do Prado
parral - nearest hotel: Hotel Adriana
rakops - nearest hotel: Rakops River Lodge
new mirpur - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
12,sao fidelis,BR,-21.6461,-41.7469,82,Hotel São José
165,toliara,MG,-23.3500,43.6667,71,Ambary
185,turpan,CN,42.9333,89.1667,15,Oriental
218,gebeit,SD,21.0667,36.3167,25,Port Sudan
330,prado,BR,-17.3411,-39.2208,79,Pousada Recanto do Prado
372,parral,MX,26.9333,-105.6667,29,Hotel Adriana
413,rakops,BW,-21.0167,24.3333,25,Rakops River Lodge
475,new mirpur,PK,33.1418,73.7674,44,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
map_with_hotel_info = hotel_df.hvplot.points('Lng', 'Lat', 
                                             hover_cols=['City', 'Country', 'Hotel Name'], 
                                             size='Humidity', color='blue', 
                                             title='City Humidity with Hotel Information',
                                             frame_height=400, frame_width=700)


# Display the map
map_with_hotel_info

:Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)